# Práctica I - Módulo I - Diplomado Ciencia de Datos  
### Hernández González Ricardo Paramont  
  
**DataSets**:  
- Parte 1: "Chat de WhatsApp con Luis Zepeda.txt"  
- Parte 2: "BOLSAA.MX.csv"

***

In [1]:
# bibliotecas
import pandas as pd
import numpy as np
import unicodedata
import re

from sklearn.model_selection import train_test_split
import nltk
from unidecode import unidecode
from textblob import TextBlob
import spacy
from collections import Counter

In [2]:
# funciones
def clean_text(text, pattern="[^a-zA-Z0-9 ]"):
    """Cleans the text to facilitate its analysis, it changes accented letters to its quivalent without accents, removes also special characters

    Parameters
    ----------
    text : string
        String containing text
    pattern : str, optional
        Regular expession for keeping in the string, by default ``[^a-zA-Z0-9 ]``

    Returns
    -------
    cleaned_text : string
        Cleaned string

    Example
    -------
    >>> clean_text('¡Feliz año nuevo, México!')
    >>> u'feliz ano nuevo mexico'
    """
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, " ", cleaned_text.decode("utf-8"), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().strip().split())
    return cleaned_text

def transform_hours(text, pattern='a.\xa0m.'):
    """
    Transforms hours from strings to integer, where their string format is divided
    in a.m. or p.m. daytime.
    
    Parameters
    ----------
    text : string 
        String representation of daytime.
    pattern : string
        String pattern that indicates if daytime is of a.m.
        
    Returns
    -------
    Hour as int in 24h format
    
    Example
    >>>transform_hours('12:40 a.\xa0m.')
    >>>0
    """
    hour_search = int(re.search('(\d{1,2})', text).group(1))
    
    if pattern in text:
        if hour_search == 12:
            return 0
        else:
            return hour_search
    else:
        if hour_search == 12:
            return 12
        else:
            return hour_search+12

***
## Parte 1: Ingeniería de variables de texto.
***

 ### 1. Exporta una conversación entre dos personas de Whatsapp. 
   
***

In [3]:
# Importando conversación.
df = pd.read_csv("Chat de WhatsApp con Luis Zepeda.txt", header=0, names=["d_fecha", "_", "__"])
# Eliminando columnas vacías.
df = df.drop(df.columns[1:3], axis=1)
df.head()

,d_fecha
0,28/08/17 8:40 a. m. - Luis Zepeda: Qué onda pa...
1,28/08/17 8:40 a. m. - Luis Zepeda: Soy Luis jaja
2,28/08/17 8:40 a. m. - Luis Zepeda: Ya llego el...
3,28/08/17 8:45 a. m. - Luis Zepeda: We?
4,28/08/17 11:56 a. m. - Paramont Hernàndez: Ah ...


In [4]:
df.shape

(1751, 1)

### 2. El documento con extensión .txt transformar en un DataFrame de la siguienteforma:

 <div style="text-align: justify"> 
<br\>
• ”t fecha” es la fecha en la que el mensaje fue enviado , en un formato de año-mes-dı́a <br\>
• ”v hora” es la hora en que el mensaje fue enviado , debe estar en un formato de 24 horas<br\>
• ”t texto” es el contenido de cada mensaje enviado <br\>
• ”v usuario” es el nombre de la persona que mando el mensaje
Aspectos importantes a considerar: <br\>
• La nomenclatura de las variables debe ser la misma. <br\>
• De acuerdo a la configuración de tu reloj, la hora puede venir en un formato
de 12 o 24 horas. Si se tiene en un formato de 12 horas es necesario hacer uso
de la información proporcionada en tu txt que indique si es ”am” o ”pm”.
    <div>
        
***        

In [5]:
# Separando las columnas según su separador específico:
df[['d_fecha','v_hora']] = df['d_fecha'].str.split(' ',1, expand=True)
df[['v_hora','t_texto']] = df['v_hora'].str.split(' - ',1, expand=True)
df[['v_usuario','t_texto']] = df['t_texto'].str.split(': ',1, expand=True)
df = df[['d_fecha','v_hora','t_texto','v_usuario']]
df.head()

,d_fecha,v_hora,t_texto,v_usuario
0,28/08/17,8:40 a. m.,Qué onda paramont,Luis Zepeda
1,28/08/17,8:40 a. m.,Soy Luis jaja,Luis Zepeda
2,28/08/17,8:40 a. m.,Ya llego el profe?,Luis Zepeda
3,28/08/17,8:45 a. m.,We?,Luis Zepeda
4,28/08/17,11:56 a. m.,Ah perdón,Paramont Hernàndez


In [6]:
# transformando d_date
df['d_fecha'] = pd.to_datetime(df['d_fecha'])
df['d_fecha']

0      2017-08-28
1      2017-08-28
2      2017-08-28
3      2017-08-28
4      2017-08-28
          ...    
1746   2020-10-05
1747   2020-10-05
1748   2020-10-05
1749   2020-10-05
1750   2020-10-05
Name: d_fecha, Length: 1751, dtype: datetime64[ns]

In [6]:
# transformando v_horas usando la función definida en el notebook.
df['v_hora'] = df['v_hora'].map(lambda hora:transform_hours(hora))
df['v_hora']

0        8
1        8
2        8
3        8
4       11
        ..
1746    15
1747    15
1748    15
1749    15
1750    17
Name: v_hora, Length: 1751, dtype: int64

### 3. Una vez teniendo la tabla en esa estructura procedemos a hacer lo siguiente:
 <div style="text-align: justify"> 
<br\>
(a) Haz el conteo de missings por variable y crea un dataframe con el siguiente formato y guardarlo en una variable llamada ”missings”. En caso de tener ”missings” eliminar los registros de tu DataFrame.<br\>
(b) Posteriormente haz el conteo de registros que tienen como valor ”Multimedia omi-tido” en la variable ”t texto” y asignala a una variable llamada ”total multimedia”, adicional a lo anterior elimina dichos registros. El número de registros mı́nimos para poder trabajar los siguientes pasos deberı́a ser 200.<br\>
(c) Verifica la calidad de los datos.<br\>
(d) Realiza la limpieza de la variable ”v usuario” , solo debes tener dos categorı́as en esta columna, el texto no debe contener caracteres especiales , en minúsculas y sin
espacios<br\>
(e) La variable ”d fecha” conviertela a tipo ”datetime”<br\>
(f) La variable ”v hora” conviertela a tipo entero<br\>
(g) Realiza la limpieza de la variable ”t texto”<br\>
(h) Elimina stop words<br\>
(i) Elimina hapaxes<br\>
(j) Realiza tokenización<br\>
(k) Realiza derivación (Stemmer) en español, se puede utilizar de la siguiente forma :<br\>
from nltk.stem import SnowballStemmer<br\>
spanish stemmer = SnowballStemmer(’spanish’)<br\>
(spanish stemmer.stem(”texto en str”))<br\>
(l) Creación de variables(Se deben crear al menos 10 nuevas variables):<br\>
Utilice Count vectorizer o Tfidf Vectorizer para generar carácteristicas además de
uno de los métodos vistos en clase para generar nuevas carácteristicas sobre el
texto(longitud , análisis de sentimientos,reconocimiento de entidad).<br\><br\>
Si se utiliza análisis de sentimientos se deberı́a hacer lo siguiente:<br\>
Primero se deberı́a traducir el texto a inglés, utilizando TextBlob
traducir = lambda x: TextBlob(x).translate(to=”en”)<br\>
Como paso siguiente se utilizará TextBlob(x).sentiment.polarity<br\>
<br\>
NOMBRE DE LA TABLA FINAL : df text
<div>
    
***

**A)** Haz el conteo de missings por variable y crea un dataframe con el siguiente formato y guardarlo en una variable llamada ”missings”. En caso de tener ”missings” eliminar los registros de tu DataFrame.

In [7]:
# Encontrando valores faltantes y guardadndolos.
missings = pd.DataFrame(df.isnull().sum(axis=0))
missings.reset_index().rename(columns={'index':'nombre_columna', 0:'total_missings'})

,nombre_columna,total_missings
0,d_fecha,0
1,v_hora,0
2,t_texto,2
3,v_usuario,0


In [8]:
# Eliminando valores faltantes.
df = df[~df['t_texto'].isnull()]
df = df.reset_index(drop=True)

**B)** Posteriormente haz el conteo de registros que tienen como valor ”Multimedia omi-tido” en la variable ”t texto” y asignala a una variable llamada ”total multimedia”, adicional a lo anterior elimina dichos registros. El número de registros mı́nimos para poder trabajar los siguientes pasos deberı́a ser 200.

In [9]:
# Bucando textos con multimedia y guardandolos
total_multimedia = df['t_texto'][df['t_texto']=='<Multimedia omitido>'].count()
total_multimedia

70

In [10]:
# Borrando dichos registros.
df = df[~(df['t_texto']=='<Multimedia omitido>')]
df = df.reset_index(drop=True)

**C)** Verifica la calidad de los datos.

La mayoría del trabajo de calidad de los datos se realiza en los siguientes incisos. Por lo que en este inciso sólo se verificará el tipo de dato de las columnas.

In [11]:
# verificando tipo de dato
df.dtypes

d_fecha      datetime64[ns]
v_hora                int64
t_texto              object
v_usuario            object
dtype: object

In [12]:
# para t_texto
df['t_texto'].map(type).value_counts()

<class 'str'>    1679
Name: t_texto, dtype: int64

In [13]:
# para v_usuario
df['v_usuario'].map(type).value_counts()

<class 'str'>    1679
Name: v_usuario, dtype: int64

**D)** Realiza la limpieza de la variable ”v usuario” , solo debes tener dos categorı́as en esta columna, el texto no debe contener caracteres especiales , en minúsculas y sin
espacios

In [14]:
# Se confirman sólo dos categorías
np.unique(df['v_usuario'])

array(['Luis Zepeda', 'Paramont Hernàndez'], dtype=object)

In [15]:
# Limpiando las strings
df["v_usuario"] = df["v_usuario"].map(lambda x:clean_text(x).replace(" ",""))

In [16]:
# Comprobando
np.unique(df['v_usuario'])

array(['luiszepeda', 'paramonthernandez'], dtype=object)

**E)** La variable ”d fecha” conviertela a tipo ”datetime”.

Durante la parte 2 se transformó a la variable d_fecha a tipo datetime para poder cumplir con el formato indicado.

In [17]:
# Se procede a confirmar que la variable es de tipo datetime:
df.dtypes

d_fecha      datetime64[ns]
v_hora                int64
t_texto              object
v_usuario            object
dtype: object

**F)** La variable ”v hora” conviertela a tipo entero.

Durante la parte 2 se transformó a la variable v_hora a tipo entero para poder cumplir con el formato indicado.

In [18]:
# Se procede a confirmar que la variable es de tipo entero:
df.dtypes

d_fecha      datetime64[ns]
v_hora                int64
t_texto              object
v_usuario            object
dtype: object

In [19]:
# Revisando si la variable efectivamente es discreta entre 0h y 23h:
# No hay valores entre 1 y 4 porque no hay mensajes en esas horas.
np.unique(df['v_hora'])

array([ 0,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23])

**G)** Realiza la limpieza de la variable ”t texto”

In [20]:
# Creando una columna donde conservaremos texto original para 
#    la creación de características.
df['t_texto_original'] = df['t_texto']

In [21]:
# Revisando ejemplo.
df['t_texto'][0]

'Qué onda paramont'

In [22]:
# limpiando t_texto
df["t_texto"] = df["t_texto"].map(lambda x:clean_text(x, pattern="[^a-zA-Z ]"))
# verificando en ejemplo
df['t_texto'][0]

'que onda paramont'

**H)** Elimina stop words.

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/paramont/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
stop_words=nltk.corpus.stopwords.words("spanish")
df["t_texto"] = df["t_texto"].map(lambda text:" ".join([x for x in text.split(" ") if x not in stop_words]))
# Revisando mismo ejemplo
df['t_texto'][0]

'onda paramont'

**I)** Elimina hapaxes.

In [25]:
# Uniendo todas las filas de testo en un solo texto.
corpus = " ".join(df["t_texto"].values)
# Usando nltk
fdist = nltk.FreqDist(corpus.split())
# obteniendo hapaxes
list_hapaxes = fdist.hapaxes()
# eliminando hapaxes
df["t_texto"] = df["t_texto"].map(lambda text:" ".join([x for x in text.split(" ") if x not in list_hapaxes]))
# Comparando con texto irginal
df['t_texto_original'][5]

'No tenía internet jeje'

In [26]:
# Se eliminan "tenia" e "internet" por ser hapaxes
df['t_texto'][5]

'jeje'

In [27]:
# Guardamos una copia del texto limpio, sin stop words ni hapaxes para 
#    count vectorizer.
df['t_texto_limpio'] = df['t_texto']
df['t_texto_limpio']

0       onda paramont
1           luis jaja
2         llego profe
3                  we
4           ah perdon
            ...      
1674     estan juntos
1675                 
1676       solo quede
1677                 
1678          chingon
Name: t_texto_limpio, Length: 1679, dtype: object

**J)** Realiza tokenización

In [28]:
df["t_texto"] = df["t_texto"].map(lambda text:text.split())
# comprobando
df["t_texto"]

0       [onda, paramont]
1           [luis, jaja]
2         [llego, profe]
3                   [we]
4           [ah, perdon]
              ...       
1674     [estan, juntos]
1675                  []
1676       [solo, quede]
1677                  []
1678           [chingon]
Name: t_texto, Length: 1679, dtype: object

**K)** Realiza derivación (Stemmer) en español, se puede utilizar de la siguiente forma :  
from nltk.stem import SnowballStemmer  
spanish stemmer = SnowballStemmer(’spanish’)  
(spanish stemmer.stem(”texto en str”))  

In [29]:
spanish_stemmer = nltk.stem.SnowballStemmer('spanish')
df['t_texto'] = df["t_texto"].map(lambda text:[spanish_stemmer.stem(word) for word in text])
# Comprobando
df.iloc[2]['t_texto']

['lleg', 'prof']

**L)** Creación de variables(Se deben crear al menos 10 nuevas variables):  
Utilice Count vectorizer o Tfidf Vectorizer para generar carácteristicas además de uno de los métodos vistos en clase para generar nuevas carácteristicas sobre el texto(longitud , análisis de sentimientos,reconocimiento de entidad).

**Longitud**

In [30]:
# número de palabras
df['c_numero_palabras'] = df['t_texto'].map(lambda text:len(text))
# número de letras por mensaje
df['c_numero_letras'] = df['t_texto'].map(lambda text:sum(len(word) for word in text))
# número promedio de palabras por mensaje
df['c_long_prom_palabra'] = df['c_numero_letras']/df['c_numero_palabras']
# Revisando
df[['c_numero_palabras','c_numero_letras','c_long_prom_palabra']]

,c_numero_palabras,c_numero_letras,c_long_prom_palabra
0,2,12,6.0
1,2,7,3.5
2,2,8,4.0
3,1,2,2.0
4,2,8,4.0
...,...,...,...
1674,2,9,4.5
1675,0,0,NaN
1676,2,7,3.5
1677,0,0,NaN


**Reconocimiento de entidad designada**

In [31]:
ner = spacy.load('es_core_news_sm')
# obteniendo entidades como diccionarios
df["entidades_designadas"] = df["t_texto_original"].map(lambda text:ner(text))
df["entidades_designadas"] = df["entidades_designadas"].map(lambda x:Counter([y.label_ for y in x.ents]))
df["entidades_designadas"]

0        {'PER': 1}
1        {'PER': 1}
2                {}
3       {'MISC': 1}
4                {}
           ...     
1674             {}
1675             {}
1676             {}
1677             {}
1678             {}
Name: entidades_designadas, Length: 1679, dtype: object

**Count vectorizer**

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
# Obteniendo Conjuntos de entrenamiento y prueba.
X_train, X_test = train_test_split(df, test_size=0.20, random_state=42)
# Definiendo vectorización para palabras que aparecen al menos 2.5% de las veces
vect = CountVectorizer(ngram_range=(1, 5), min_df=0.025)
# Entrenando modelo
vect.fit(X_train["t_texto_limpio"])

CountVectorizer(min_df=0.025, ngram_range=(1, 5))

In [33]:
#Se transforman los datos de train para obtener el conteo por palabra
array_train = vect.transform(X_train["t_texto_limpio"])
array_test = vect.transform(X_test["t_texto_limpio"])
aux_train = pd.DataFrame(array_train.toarray(),columns=vect.get_feature_names())
aux_test = pd.DataFrame(array_test.toarray(),columns=vect.get_feature_names())

In [34]:
# Insertando la vectorización en el df.
X_train = X_train.reset_index()
X_test = X_test.reset_index()
X_train = pd.concat([X_train,aux_train],axis=1)
X_test = pd.concat([X_test,aux_test],axis=1)
df_text_train = X_train.set_index('index').sort_index()
df_text_test = X_test.set_index('index').sort_index()

**NOMBRE DE LA TABLA FINAL : df text**

In [35]:
df_text_train

,d_fecha,v_hora,t_texto,v_usuario,t_texto_original,t_texto_limpio,c_numero_palabras,c_numero_letras,c_long_prom_palabra,entidades_designadas,bien,jaja,jajaja,jajajaja,pues,si
index,,,,,,,,,,,,,,,,
0,2017-08-28,8,"[onda, paramont]",luiszepeda,Qué onda paramont,onda paramont,2,12,6.0,{'PER': 1},0,0,0,0,0,0
1,2017-08-28,8,"[luis, jaj]",luiszepeda,Soy Luis jaja,luis jaja,2,7,3.5,{'PER': 1},0,1,0,0,0,0
2,2017-08-28,8,"[lleg, prof]",luiszepeda,Ya llego el profe?,llego profe,2,8,4.0,{},0,0,0,0,0,0
3,2017-08-28,8,[we],luiszepeda,We?,we,1,2,2.0,{'MISC': 1},0,0,0,0,0,0
4,2017-08-28,11,"[ah, perdon]",paramonthernandez,Ah perdón,ah perdon,2,8,4.0,{},0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1674,2020-10-05,15,"[estan, junt]",paramonthernandez,Están juntos?,estan juntos,2,9,4.5,{},0,0,0,0,0,0
1675,2020-10-05,15,[],luiszepeda,No,,0,0,NaN,{},0,0,0,0,0,0
1676,2020-10-05,15,"[sol, qued]",luiszepeda,Solo yo quedé,solo quede,2,7,3.5,{},0,0,0,0,0,0


In [36]:
df_text_test

,d_fecha,v_hora,t_texto,v_usuario,t_texto_original,t_texto_limpio,c_numero_palabras,c_numero_letras,c_long_prom_palabra,entidades_designadas,bien,jaja,jajaja,jajajaja,pues,si
index,,,,,,,,,,,,,,,,
15,2017-10-22,12,"[hol, we]",paramonthernandez,Hola we,hola we,2,5,2.500000,{'PER': 1},0,0,0,0,0,0
23,2017-10-22,12,"[da, coeficient]",paramonthernandez,Pero lo que me da miedo son los coeficientes d...,da coeficientes,2,12,6.000000,{},0,0,0,0,0,0
29,2017-10-22,12,[ejercici],paramonthernandez,Esto es del ejercicio 4,ejercicio,1,8,8.000000,{},0,0,0,0,0,0
30,2017-10-22,12,"[g, mal]",paramonthernandez,Pero la g) la tenemos mal,g mal,2,4,2.000000,{},0,0,0,0,0,0
32,2017-10-22,12,"[calcul, g]",paramonthernandez,Porque hay que calcular Delta G del cloruro,calcular g,2,7,3.500000,{'ORG': 1},0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1664,2020-10-05,15,[pas],luiszepeda,Pasaron*,pasaron,1,3,3.000000,{},0,0,0,0,0,0
1666,2020-10-05,15,"[despu, van, entrev]",luiszepeda,Ya después de eso van las entrevistas,despues van entrevistas,3,14,4.666667,{},0,0,0,0,0,0
1667,2020-10-05,15,[jaj],luiszepeda,Jaja,jaja,1,3,3.000000,{'LOC': 1},0,1,0,0,0,0


***
## Parte 2: Ingeniería de variables continuas.
***

<div style="text-align: justify"> 
Con el conjunto de datos ”BOLSAA.MX.csv” realice las siguientes actividades:<br\>
(a) Etiquetado de las variables<br\>
(b) Calidad de datos, para verificar que no existan anomalı́as<br\>
(c) Verifique que los datos estén ordenados por fecha<br\>
(d) ¿Cuál es la diferencia promedio entre fechas?<br\>
(e) (TABLA 1) Generé las siguientes variables sin cambiar la frecuencia de los registros
(con fechas originales, sin agrupaciones):<br\>
• Realice el desfase de la variable ”Cierre” de tal forma que los valores por
registro le corresponda el valor de ”Cierre” de dos periodos posteriores<br\>
• Variable ”diff max min” que sea la diferencia de los valores High con los valores
Low<br\>
• Variable ”pct cie ape” cambio porcentual entre el valor de cierre y apertura<br\>
• Variable ”ratio” que corresponda a los valores de cierre entre los valores de
apertura<br\>
• Cree las siguientes variables para esta lista: [’Open’, ’High’, ’Low’, ’Volume’]<br\>
-Columnas de diferencia de 1 periodo a 3 perı́odos para cada variable<br\>
-Columnas de ventana de tiempo de 1 ,..., 3 para cada variable utilizando la
media<br\>
-Columnas de cambio porcentual de 1 periodo a 3 perı́odos para cada variable<br\>
-Generar el cuadrado de cada variable<br\>
(f) (TABLA 2) Cambie la frecuencia de las fechas de tal forma que los registros sean
cada 3 dı́as y genere al menos cuatro nuevas variables por cada variable original.<br\>
No olvide hacer el desfase de la variable objetivo.<br\>
<div>
    
***

In [37]:
df2 = pd.read_csv("BOLSAA.MX.csv")

**A)** Etiquetado de las variables.

In [38]:
df2 = df2.rename(columns={'Open':'c_open', 'High':'c_high', 'Low':'c_low', 'Close':'c_close', 'Adj Close':'c_adj_close', 'Volume':'c_volume', 'date':'d_date'})

**B)** Calidad de datos, para verificar que no existan anomalı́as

In [39]:
# buscando registros nulos
missings = pd.DataFrame(df2.isnull().sum(axis=0))
missings

,0
c_open,0
c_high,0
c_low,0
c_close,0
c_adj_close,0
c_volume,0
d_date,0


In [40]:
# buscando duplicados
df2.duplicated().sum()

0

In [41]:
# verificando datatype
df2.dtypes

c_open         float64
c_high         float64
c_low          float64
c_close        float64
c_adj_close    float64
c_volume         int64
d_date          object
dtype: object

In [42]:
# revisando d_date
df2['d_date'].map(type).value_counts()

<class 'str'>    5040
Name: d_date, dtype: int64

In [43]:
# cambiando d_date a datetime
df2['d_date'] = pd.to_datetime(df2['d_date'])

In [44]:
# Verificando que la primer columna es una variable continua
df2['c_open'].value_counts()

42.000000    100
42.669998     80
43.520000     60
41.500000     60
42.590000     40
            ... 
40.820000     20
45.209999     20
42.700001     20
46.139999     20
46.400002     20
Name: c_open, Length: 218, dtype: int64

**C)** Verifique que los datos estén ordenados por fecha.

In [45]:
df2['d_date'].equals(df2['d_date'].sort_values())

True

In [46]:
# Se confirma que el orden original de d_date es el correcto.

**D)** ¿Cuál es la diferencia promedio entre fechas?

In [47]:
avg_date_diff = df2['d_date'].diff().sum()/(len(df2)-1)
avg_date_diff

Timedelta('1 days 00:18:06.346497320')

In [48]:
df2['d_date'].diff().describe()

count                         5039
mean     1 days 00:18:06.346497320
std      0 days 00:08:25.902877163
min                1 days 00:04:00
25%                1 days 00:11:00
50%                1 days 00:18:00
75%                1 days 00:26:00
max                1 days 00:32:00
Name: d_date, dtype: object

**E)** (TABLA 1) Generé las variables previamente indicadas sin cambiar la frecuencia de los registros (con fechas originales, sin agrupaciones).

• Realice el desfase de la variable ”Cierre” de tal forma que los valores por
registro le corresponda el valor de ”Cierre” de dos periodos posteriores.

In [49]:
tabla_1 = pd.DataFrame()
tabla_1['c_close'] = df2['c_close'].shift(2)
tabla_1

,c_close
0,NaN
1,NaN
2,39.880001
3,40.000000
4,40.790001
...,...
5035,45.730000
5036,46.099998
5037,46.099998
5038,46.240002


• Variable ”diff max min” que sea la diferencia de los valores High con los valores
Low.

In [50]:
tabla_1['diff_max_min'] = df2['c_high']-df2['c_low']
tabla_1

,c_close,diff_max_min
0,NaN,2.060002
1,NaN,1.549999
2,39.880001,2.439999
3,40.000000,1.610000
4,40.790001,1.319999
...,...,...
5035,45.730000,0.739998
5036,46.099998,0.900002
5037,46.099998,1.070000
5038,46.240002,1.139999


• Variable ”pct cie ape” cambio porcentual entre el valor de cierre y apertura.

In [51]:
tabla_1['pct_cie_ape'] = df2['c_close']/(df2['c_open']*100)
tabla_1

,c_close,diff_max_min,pct_cie_ape
0,NaN,2.060002,0.010415
1,NaN,1.549999,0.010030
2,39.880001,2.439999,0.010249
3,40.000000,1.610000,0.010218
4,40.790001,1.319999,0.010029
...,...,...,...
5035,45.730000,0.739998,0.009957
5036,46.099998,0.900002,0.009938
5037,46.099998,1.070000,0.009959
5038,46.240002,1.139999,0.010022


• Variable ”ratio” que corresponda a los valores de cierre entre los valores de
apertura.

In [52]:
tabla_1['ratio'] = tabla_1['pct_cie_ape']*100
tabla_1

,c_close,diff_max_min,pct_cie_ape,ratio
0,NaN,2.060002,0.010415,1.041525
1,NaN,1.549999,0.010030,1.003009
2,39.880001,2.439999,0.010249,1.024874
3,40.000000,1.610000,0.010218,1.021787
4,40.790001,1.319999,0.010029,1.002871
...,...,...,...,...
5035,45.730000,0.739998,0.009957,0.995680
5036,46.099998,0.900002,0.009938,0.993768
5037,46.099998,1.070000,0.009959,0.995890
5038,46.240002,1.139999,0.010022,1.002167


• Cree las siguientes variables para esta lista: [’Open’, ’High’, ’Low’, ’Volume’]
  
-Columnas de diferencia de 1 periodo a 3 perı́odos para cada variable  
-Columnas de ventana de tiempo de 1 ,..., 3 para cada variable utilizando la
media  
-Columnas de cambio porcentual de 1 periodo a 3 perı́odos para cada variable  
-Generar el cuadrado de cada variable  

In [53]:
# columnas de diferencia
# columnas de media
# columnas de cambio porcentual
# cuadrado de las variables generadas
columns = ['c_open', 'c_high', 'c_low', 'c_volume']
for col in columns:
    for i in range(3):
        tabla_1[f"{col}_diff_{i+1}"] = df2[col].diff(i+1)
        tabla_1[f"{col}_ma_{i+1}"] = df2[col].rolling(i+2).mean()
        tabla_1[f"{col}_pct_{i+1}"] = df2[col].pct_change(i+1)
    tabla_1[f"{col}_sq"] = df2[col].map(lambda x:x**2)
tabla_1

,c_close,diff_max_min,pct_cie_ape,ratio,c_open_diff_1,c_open_ma_1,c_open_pct_1,c_open_diff_2,c_open_ma_2,c_open_pct_2,...,c_volume_diff_1,c_volume_ma_1,c_volume_pct_1,c_volume_diff_2,c_volume_ma_2,c_volume_pct_2,c_volume_diff_3,c_volume_ma_3,c_volume_pct_3,c_volume_sq
0,NaN,2.060002,0.010415,1.041525,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9382569357604
1,NaN,1.549999,0.010030,1.003009,1.590000,39.085001,0.041525,NaN,NaN,NaN,...,-2593188.0,1766504.0,-0.846590,NaN,NaN,NaN,NaN,NaN,NaN,220815408100
2,39.880001,2.439999,0.010249,1.024874,-0.080002,39.840000,-0.002006,1.509998,39.323334,0.039436,...,5753139.0,3346479.5,12.243066,3159951.0,3.252019e+06,1.031619,NaN,NaN,NaN,38726338856401
3,40.000000,1.610000,0.010218,1.021787,1.049999,40.324998,0.026382,0.969997,40.176666,0.024323,...,-2658887.0,4893605.5,-0.427264,3094252.0,3.419040e+06,6.584776,501064.0,3330054.75,0.163581,12703250762244
4,40.790001,1.319999,0.010029,1.002871,0.950001,41.324998,0.023256,2.000000,40.816665,0.050251,...,-1653209.0,2737557.5,-0.463842,-4312096.0,3.899388e+06,-0.692923,1441043.0,3042018.50,3.066636,3651741368209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5035,45.730000,0.739998,0.009957,0.995680,0.799999,45.899999,0.017582,-0.890000,46.329999,-0.018860,...,-1363130.0,1085499.0,-0.771410,-332063.0,9.689983e+05,-0.451174,-330869.0,910449.50,-0.450283,163162676356
5036,46.099998,0.900002,0.009938,0.993768,0.230000,46.414999,0.004968,1.029999,46.109999,0.022637,...,133958.0,470913.0,0.331633,-1229172.0,9.029633e+05,-0.695601,-198105.0,861221.75,-0.269165,289327803664
5037,46.099998,1.070000,0.009959,0.995890,-0.299999,46.379999,-0.006447,-0.069999,46.353333,-0.001512,...,1292501.0,1184142.5,2.402901,1426459.0,9.240730e+05,3.531416,63329.0,1134820.75,0.035839,3350338534449
5038,46.240002,1.139999,0.010022,1.002167,-0.090001,46.185000,-0.001947,-0.390000,46.299999,-0.008382,...,557428.0,2109107.0,0.304540,1849929.0,1.585369e+06,3.439220,1983887.0,1290010.00,4.911414,5701689128041


In [55]:
# Obteniendo igual nuevas variables por la creación de sus polinomios de grado 2.
# En caso de que a eso se haya referido el último punto de e)
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)

In [56]:
tabla_1_poly = poly.fit_transform(df2[columns])
tabla_1_poly = pd.DataFrame(columns=poly.get_feature_names(), data=tabla_1_poly)
tabla_1_poly.head()

,1,x0,x1,x2,x3,x0^2,x0 x1,x0 x2,x0 x3,x1^2,x1 x2,x1 x3,x2^2,x2 x3,x3^2
0,1.0,38.290001,40.160000,38.099998,3063098.0,1466.124177,1537.726440,1458.848962,1.172860e+08,1612.825600,1530.095920,1.230140e+08,1451.609848,1.167040e+08,9.382569e+12
1,1.0,39.880001,40.139999,38.590000,469910.0,1590.414480,1600.783200,1538.969239,1.874001e+07,1611.219520,1549.002561,1.886219e+07,1489.188100,1.813383e+07,2.208154e+11
2,1.0,39.799999,41.959999,39.520000,6223049.0,1584.039920,1670.007918,1572.895960,2.476773e+08,1760.641516,1658.259160,2.611191e+08,1561.830400,2.459349e+08,3.872634e+13
3,1.0,40.849998,42.299999,40.689999,3564162.0,1668.722337,1727.954875,1662.186378,1.455960e+08,1789.289915,1721.186917,1.507640e+08,1655.676019,1.450257e+08,1.270325e+13
4,1.0,41.799999,42.520000,41.200001,1910953.0,1747.239916,1777.335957,1722.160001,7.987783e+07,1807.950400,1751.824043,8.125372e+07,1697.440082,7.873127e+07,3.651741e+12


**F)** (TABLA 2) Cambie la frecuencia de las fechas de tal forma que los registros sean cada 3 dı́as y genere al menos cuatro nuevas variables por cada variable original.
No olvide hacer el desfase de la variable objetivo.

In [57]:
from scipy.stats import kurtosis 
from scipy.stats import skew

In [58]:
##AGGREGATION FUNCTIONS
def minPctChange(x):
    return x.pct_change().replace([np.inf, -np.inf], np.nan).min()
def maxPctChange(x):
    return x.pct_change().replace([np.inf, -np.inf], np.nan).max()
def meanPctChange(x):
    return x.pct_change().replace([np.inf, -np.inf], np.nan).mean()
def stdPctChange(x):
    return x.pct_change().replace([np.inf, -np.inf], np.nan).std()
def sumPctChange(x):
    return x.pct_change().replace([np.inf, -np.inf], np.nan).sum()
def minChange(x):
    return x.diff().replace([np.inf, -np.inf], np.nan).min()
def maxChange(x):
    return x.diff().replace([np.inf, -np.inf], np.nan).max()
def meanChange(x):
    return x.diff().replace([np.inf, -np.inf], np.nan).mean()
def stdChange(x):
    return x.diff().replace([np.inf, -np.inf], np.nan).std()
def sumChange(x):
    return x.diff().replace([np.inf, -np.inf], np.nan).sum()

In [59]:
# Obteniendo Variables continuas
variables=(tabla_1.filter(like="c_").columns)
len(variables)

41

In [60]:
# Utilizando dataframe original.
df_aux=df2.copy()
diff = list(filter(lambda x:"diff_" in x, df_aux.columns))
ma = list(filter(lambda x:"ma_" in x, df_aux.columns))
pct = list(filter(lambda x:"pct_" in x, df_aux.columns))
sq = list(filter(lambda x:"sq_" in x, df_aux.columns))

In [61]:
aux = diff+ma+pct+sq
df_aux=df_aux.dropna(subset=aux)
df_aux.isnull().sum()

c_open         0
c_high         0
c_low          0
c_close        0
c_adj_close    0
c_volume       0
d_date         0
dtype: int64

In [62]:
from scipy.stats import kurtosis 
from scipy.stats import skew

In [63]:
df2.set_index("d_date",inplace=True)

In [64]:
X=df2[[col for col in df2 if col!="c_close"]]
y=df2[["c_close"]]

In [65]:
tgt=y.resample("3D").mean()
X_aux=X.resample("3D").agg(["min", "max", "mean", "sum", "std", "median",kurtosis,skew,minPctChange,sumChange])
X_aux.columns = [ name+" | "+func for name, func in X_aux.columns]
df_agg=pd.concat([X_aux,tgt.shift(-1)],axis=1)
df_agg.head()

,c_open | min,c_open | max,c_open | mean,c_open | sum,c_open | std,c_open | median,c_open | kurtosis,c_open | skew,c_open | minPctChange,c_open | sumChange,...,c_volume | max,c_volume | mean,c_volume | sum,c_volume | std,c_volume | median,c_volume | kurtosis,c_volume | skew,c_volume | minPctChange,c_volume | sumChange,c_close
d_date,,,,,,,,,,,,,,,,,,,,,
2004-12-01,38.290001,39.880001,39.323334,117.970001,0.895786,39.799999,-1.5,-0.700767,-0.002006,1.509998,...,6223049,3.252019e+06,9756057,2.881219e+06,3063098.0,-1.5,0.119942,-0.846590,3159951,41.946667
2004-12-04,40.849998,42.020000,41.556666,124.669997,0.621799,41.799999,-1.5,-0.608832,0.005263,1.170002,...,3564162,2.215200e+06,6645599,1.225499e+06,1910953.0,-1.5,0.427978,-0.463842,-2393678,42.446667
2004-12-07,42.200001,43.360001,42.636667,127.910000,0.630900,42.349998,-1.5,0.662407,-0.023293,0.149997,...,1906038,1.505892e+06,4517675,6.133591e+05,1811903.0,-1.5,-0.688415,-0.558622,94135,42.630001
2004-12-10,41.970001,43.529999,42.736666,128.209999,0.780341,42.709999,-1.5,0.062708,0.017632,1.559998,...,2541907,1.428543e+06,4285628,9.740040e+05,1009696.0,-1.5,0.643918,-0.273024,1532211,41.813333
2004-12-13,41.619999,41.799999,41.713333,125.139999,0.090185,41.720001,-1.5,-0.135088,-0.002397,-0.180000,...,890572,7.101903e+05,2130571,2.907773e+05,865251.0,-1.5,-0.701079,-0.566891,-515824,42.113333


In [66]:
#Observamos explosión de variables
print(df2.shape,df_agg.shape)

(5040, 6) (1701, 51)


**NOMBRES DE LA TABLAS FINALES** : df ori, df 3 days

In [67]:
# de la df original, o sea df2
df_ori = df2
df_ori.head()

,c_open,c_high,c_low,c_close,c_adj_close,c_volume
d_date,,,,,,
2004-12-01 00:00:00,38.290001,40.160000,38.099998,39.880001,38.219864,3063098
2004-12-02 00:13:00,39.880001,40.139999,38.590000,40.000000,38.334866,469910
2004-12-03 00:38:00,39.799999,41.959999,39.520000,40.790001,39.091980,6223049
2004-12-04 01:08:00,40.849998,42.299999,40.689999,41.740002,40.002434,3564162
2004-12-05 01:21:00,41.799999,42.520000,41.200001,41.919998,40.174938,1910953


In [68]:
# de la df tras generar variables en el inciso f, o sea df_agg
df_3_days = df_agg
df_3_days.head()

,c_open | min,c_open | max,c_open | mean,c_open | sum,c_open | std,c_open | median,c_open | kurtosis,c_open | skew,c_open | minPctChange,c_open | sumChange,...,c_volume | max,c_volume | mean,c_volume | sum,c_volume | std,c_volume | median,c_volume | kurtosis,c_volume | skew,c_volume | minPctChange,c_volume | sumChange,c_close
d_date,,,,,,,,,,,,,,,,,,,,,
2004-12-01,38.290001,39.880001,39.323334,117.970001,0.895786,39.799999,-1.5,-0.700767,-0.002006,1.509998,...,6223049,3.252019e+06,9756057,2.881219e+06,3063098.0,-1.5,0.119942,-0.846590,3159951,41.946667
2004-12-04,40.849998,42.020000,41.556666,124.669997,0.621799,41.799999,-1.5,-0.608832,0.005263,1.170002,...,3564162,2.215200e+06,6645599,1.225499e+06,1910953.0,-1.5,0.427978,-0.463842,-2393678,42.446667
2004-12-07,42.200001,43.360001,42.636667,127.910000,0.630900,42.349998,-1.5,0.662407,-0.023293,0.149997,...,1906038,1.505892e+06,4517675,6.133591e+05,1811903.0,-1.5,-0.688415,-0.558622,94135,42.630001
2004-12-10,41.970001,43.529999,42.736666,128.209999,0.780341,42.709999,-1.5,0.062708,0.017632,1.559998,...,2541907,1.428543e+06,4285628,9.740040e+05,1009696.0,-1.5,0.643918,-0.273024,1532211,41.813333
2004-12-13,41.619999,41.799999,41.713333,125.139999,0.090185,41.720001,-1.5,-0.135088,-0.002397,-0.180000,...,890572,7.101903e+05,2130571,2.907773e+05,865251.0,-1.5,-0.701079,-0.566891,-515824,42.113333


***

***

Se hará ahora con escalamiento.

**MinMax**

In [69]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=43)
X=X_train

In [70]:
# la siguiente operación ajusta todos los valores a un rango entre 0 y 1
X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
X_std.describe()

,c_open,c_high,c_low,c_adj_close,c_volume
count,3780.000000,3780.000000,3780.000000,3780.000000,3780.000000
mean,0.633467,0.555474,0.637284,0.604376,0.154536
std,0.151310,0.177681,0.162723,0.160039,0.136279
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.578216,0.476718,0.596388,0.522798,0.073611
50%,0.622591,0.536031,0.638375,0.585503,0.115154
75%,0.719857,0.665604,0.729571,0.716614,0.191244
max,1.000000,1.000000,1.000000,1.000000,1.000000


**MinMax Sklearn**

In [71]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(X_train)
pd.DataFrame(scaler.transform(X_train))

,0,1,2,3,4
0,0.612730,0.572062,0.632506,0.656911,0.215707
1,0.942627,0.905765,0.879458,0.850333,0.046351
2,0.658001,0.547118,0.682167,0.588638,0.260993
3,0.645450,0.519401,0.615350,0.541218,0.094485
4,0.364859,0.269401,0.394131,0.360942,0.128797
...,...,...,...,...,...
3775,0.613178,0.490022,0.608578,0.520055,0.109102
3776,0.580009,0.455100,0.598194,0.511433,0.073611
3777,0.602869,0.499446,0.632957,0.560813,0.139246
3778,0.578216,0.465632,0.577427,0.520839,0.166368


In [72]:
X_train_m=pd.DataFrame(scaler.transform(X_train))

**Standard Scaler**

In [73]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [74]:
X_train_s=pd.DataFrame(scaler.transform(X_train))
X_test_s=pd.DataFrame(scaler.transform(X_test))

Creando tabla donde se comparan ambos escalamientos.

In [75]:
tabla_scales = pd.concat([X_std.reset_index(),X_train_m.reset_index(),X_train_s.reset_index()], axis=1, keys=['MinMax','MinMaxSklearn','Std'])
tabla_scales

MinMax                                                      \
                  d_date    c_open    c_high     c_low c_adj_close  c_volume   
0    2010-04-21 13:47:00  0.612730  0.572062  0.632506    0.656911  0.215707   
1    2006-04-10 05:47:00  0.942627  0.905765  0.879458    0.850333  0.046351   
2    2015-10-07 03:12:00  0.658001  0.547118  0.682167    0.588638  0.260993   
3    2007-05-15 02:52:00  0.645450  0.519401  0.615350    0.541218  0.094485   
4    2005-03-29 11:36:00  0.364859  0.269401  0.394131    0.360942  0.128797   
...                  ...       ...       ...       ...         ...       ...   
3775 2005-10-07 20:18:00  0.613178  0.490022  0.608578    0.520055  0.109102   
3776 2010-08-22 03:57:00  0.580009  0.455100  0.598194    0.511433  0.073611   
3777 2011-05-13 10:47:00  0.602869  0.499446  0.632957    0.560813  0.139246   
3778 2011-04-21 03:39:00  0.578216  0.465632  0.577427    0.520839  0.166368   
3779 2014-04-27 15:59:00  0.363066  0.207871  0.362980    0.316265  0.324973   

     MinMaxSklearn                                                     Std  \
             index         0         1         2         3         4 index   
0                0  0.612730  0.572062  0.632506  0.656911  0.215707     0   
1                1  0.942627  0.905765  0.879458  0.850333  0.046351     1   
2                2  0.658001  0.547118  0.682167  0.588638  0.260993     2   
3                3  0.645450  0.519401  0.615350  0.541218  0.094485     3   
4                4  0.364859  0.269401  0.394131  0.360942  0.128797     4   
...            ...       ...       ...       ...       ...       ...   ...   
3775          3775  0.613178  0.490022  0.608578  0.520055  0.109102  3775   
3776          3776  0.580009  0.455100  0.598194  0.511433  0.073611  3776   
3777          3777  0.602869  0.499446  0.632957  0.560813  0.139246  3777   
3778          3778  0.578216  0.465632  0.577427  0.520839  0.166368  3778   
3779          3779  0.363066  0.207871  0.362980  0.316265  0.324973  3779   

                                                        
             0         1         2         3         4  
0    -0.137073  0.093371 -0.029367  0.328306  0.448925  
1     2.043486  1.971723  1.488461  1.537057 -0.793953  
2     0.162161 -0.047037  0.275864 -0.098354  0.781269  
3     0.079206 -0.203047 -0.134810 -0.394698 -0.440709  
4    -1.775453 -1.610249 -1.494473 -1.521291 -0.188894  
...        ...       ...       ...       ...       ...  
3775 -0.134110 -0.368416 -0.176432 -0.526950 -0.333433  
3776 -0.353350 -0.564988 -0.240253 -0.580830 -0.593898  
3777 -0.202253 -0.315373 -0.026592 -0.272241 -0.112214  
3778 -0.365202 -0.505705 -0.367895 -0.522051  0.086834  
3779 -1.787304 -1.956590 -1.685935 -1.800491  1.250808  

[3780 rows x 18 columns]